In [1]:
from greeter_allocation import *
from register_salesfloor_acclocation import *
from utils import transform_time_inout, create_working_flag, create_remaining_hours, alert_employee_shortage 
import warnings
warnings.filterwarnings("ignore")

In [2]:
# 1. Read Emp Availability table, filter and format Time columns
path="../00_Input/01_Emp_Availability_Initial.xlsx" # TODO: Read from S3
df = pd.read_excel(path, names=['Name', 'Responsibility', 'Time in', 'Time out'])
filtered_df = df[~df['Responsibility'].isin(['Technology', 'Office Work']) & ~df['Name'].str.contains('Available')] # Filter out not-required roles and names
filtered_df= transform_time_inout(filtered_df)

# 2. Create Working Flag and Remaining Hours Left
work_status_df = create_working_flag(filtered_df)
work_status_df= work_status_df[work_status_df['Working Flag']==1] # Filter only working hours for every employee
work_status_df= create_remaining_hours(work_status_df, filtered_df)
work_status_df['Start_time'] = pd.to_datetime(work_status_df['Start_time'], format='%H:%M:%S').dt.time
work_status_df['End_time'] = pd.to_datetime(work_status_df['End_time'], format='%H:%M:%S').dt.time



 df.head()
                    Name Responsibility             Time in  \
0          Ana* Gonzalez   Lead Student 2024-10-18 08:30:00   
1          Aidan Priller    Sales Floor 2024-10-18 08:45:00   
2  Analuisa Flores Teran    Sales Floor 2024-10-18 08:45:00   
3        Caroline Lester    Sales Floor 2024-10-18 08:45:00   
4       Saaijeesh Naresh        Greeter 2024-10-18 08:45:00   

             Time out  
0 2024-10-18 10:00:00  
1 2024-10-18 10:00:00  
2 2024-10-18 11:00:00  
3 2024-10-18 10:00:00  
4 2024-10-18 13:00:00  
Columns:
Index(['Name', 'Responsibility', 'Time in', 'Time out'], dtype='object')
First Row:
Name                    Ana* Gonzalez
Responsibility           Lead Student
Time in           2024-10-18 08:30:00
Time out          2024-10-18 10:00:00
Name: 0, dtype: object
Second Row:
Name                    Aidan Priller
Responsibility            Sales Floor
Time in           2024-10-18 08:45:00
Time out          2024-10-18 10:00:00
Name: 1, dtype: object


In [3]:
#1. Read Shift Req table, format Time columns
emp_count_req= pd.read_excel('../00_Input/02_Emp_Count_Requirement.xlsx') # TODO: Read from S3
emp_count_req['From_Time'] = pd.to_datetime(emp_count_req['From_Time'], format='%H:%M:%S').dt.time
emp_count_req['To_Time'] = pd.to_datetime(emp_count_req['To_Time'], format='%H:%M:%S').dt.time

# 2. Alert if available employees are insufficient to satisfy the required count
emp_requirements= alert_employee_shortage(work_status_df, emp_count_req)

No shortage of employees for the whole day


In [4]:
greeter_assignment, greeter_shift_done_dict = allocate_greeter(work_status_df, emp_requirements)

Current Period: 07:30:00 to 08:00:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:00:00 to 08:30:00
Current Period Data:
 Empty DataFrame
Columns: [Name, Start_time, End_time, Working Flag, Remaining_hours_left, Priority]
Index: []
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
-----------------------------------------------------------------------------
Current Period: 08:30:00 to 09:00:00
Current Period Data:
             Name Start_time  End_time  Working Flag  Remaining_hours_left  \
0  Ana* Gonzalez   08:30:00  09:00:00             1                   1.5   

   Priority  
0         1  
Upstairs Greeters Assigned: []
Downstairs Greeters Assigned: []
------------------------------------------------------------------------

In [5]:
greeter_assignment

,From_Time,To_Time,Greeter_Down_Needed,Greeter_Up_Needed,Upstairs Greeter,Downstairs Greeter
0,07:30:00,08:00:00,0,0,None,None
1,08:00:00,08:30:00,0,0,None,None
2,08:30:00,09:00:00,0,0,None,None
3,09:00:00,09:30:00,0,1,Ana* Gonzalez,None
4,09:30:00,10:00:00,0,1,Aidan Priller,None
5,10:00:00,10:30:00,0,1,Analuisa Flores Teran,None
6,10:30:00,11:00:00,0,1,Saaijeesh Naresh,None
7,11:00:00,11:30:00,1,1,Caroline Lester,Ethan Palmer
8,11:30:00,12:00:00,1,1,Solymar Kneale,Marsailles Wesley
9,12:00:00,12:30:00,1,1,Om Jadhav,Elijah Kodjak


In [6]:
register_allocation= allocate_register_salesfloor(emp_requirements, work_status_df, greeter_assignment)
final_allocation= pd.merge(greeter_assignment, register_allocation, how='outer', left_on=['From_Time', 'To_Time'], right_on=['From_Time', 'To_Time'])

---------------------------------------------------------------------------------------------
Allocating registers between 07:30:00 to 08:00:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 08:00:00 to 08:30:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 08:30:00 to 09:00:00 ...
No Registers Needed
---------------------------------------------------------------------------------------------
Allocating registers between 09:00:00 to 09:30:00 ...
Total # Emp Working: 9
Total Emp Working: ['Ana* Gonzalez', 'Aidan Priller', 'Analuisa Flores Teran', 'Caroline Lester', 'Saaijeesh Naresh', 'Ethan Palmer', 'Jackson Carter', 'Solymar Kneale', 'Teddrose* Tewolde']
Greeters Count: 1
Current Greeters: ['Ana* Gonzalez']
RU_retained_count 0
RD_retained_count 0
retained_RUs []
retained_RDs []
new_RU_n

In [8]:
final_allocation.to_excel(r"..\01_Output\Final_Allocation_test1.xlsx", index=False)